In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import elephant
import quantities as pq
import neo
import sys
from scipy.stats import ks_2samp
plt.rcParams['figure.figsize'] = (14, 10)
import scipy.stats as stats
import itertools
from elephant.conversion import BinnedSpikeTrain
from elephant.spike_train_correlation import correlation_coefficient
sys.setrecursionlimit(10000)
from pathlib import Path
# If seaborn and elephant not available, run the sommands:
# !{sys.executable} -m pip install elephant
# !{sys.executable} -m pip install seaborn

Want to calculate the CC correlation between each excitatory neuron and the first inhibitory neuron. Code for retrieveing spike times from neuron 1 in inhibitory population. Remember that we first record all 10 000 excitatory neurons, so the first inhibitory neuron is the neuron with id 10 001.

In [2]:
def get_first_inh(inh):
    """
    Takes in path to dat file of inhibitory population spike times.
    
    This function retrieves the spike times of the first neuron in the inhibitory population.
    Return it as a spiketrain object.
    """
    #inh = pd.read_csv(path, skiprows=2, sep='\t')
    spike_data = inh.sort_values(by='time_ms')
    grouped = spike_data.groupby(spike_data['sender'])
    t = np.array(grouped.get_group(10001)['time_ms'])
    spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
    
    return spiketrain

In [3]:
def get_ccs(exc, first_inh, bs=5):
    """
    Get the CC between each excitatory neuron and first inhibitory neuron.
    CC = (x-mean(x))(y-mean(y))/sqrt((x-mean(x))^2(y-mean(y))^2)
    """
    # get all spike trains of excitatory neurons
    spike_data = exc.sort_values(by='time_ms')
    grouped = spike_data.groupby(spike_data['sender'])
    spike_trains = []
    for name, group in grouped:
        t = np.asarray(group['time_ms'])
        spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
        spike_trains.append(spiketrain)
    
    correlations = []
    for i in range(len(spike_trains)):
        # calculate cc matrix for the pair of spiketrains
        cc_matrix = correlation_coefficient(BinnedSpikeTrain([spike_trains[i],first_inh], bin_size=bs*pq.ms))
        # retrieve element not on diagonala
        cc = cc_matrix[1,0]
        correlations.append(cc)
        
    return correlations

In [4]:
def get_all_ccs(exc,inh,bs=5):
    """
    This function takes in two nested lists. exc is list of tables of spike times for excitatory popuulation.
    Same for inh but inhibitory population.
    """
    cc_list = list()
    for i in range(1,11):
        first_inh = get_first_inh(inh[i])
        exc_ccs = get_ccs(exc[i],first_inh,bs=bs)
        cc_list.append(exc_ccs)
    return cc_list

What we need to do, is model by model retrieve all excitatory and inhibitory spike time files, save in list of tables. Then send these into the get_all_ccs function to retrieve cc list for all seed values and all neurons. Remember to do this for all .

All files from rounded spike times and delay 1.0 to 2.0 have same file names, can use function below to retrieve the data.

In [5]:
def round_1_2(spike_path):
    """
    Get spike data from brunel model with rounded spike times and delays drawn from discrete interval [1.0, 2.0].
    Send in any path we need.
    """
    #spike_path = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_2\\brunel_rounding_1_2'
    exc = {}
    for i in range(1,11):
        exc[i] = pd.read_csv(r'{}/brunel_rounding_True_delay_1.0_2.0_seed_{}_spikes_exc-12502-0.dat'.format(spike_path, i),
                             skiprows=2,sep='\t')

    inh = {}
    for i in range(1,11):
        inh[i] = pd.read_csv(r'{}/brunel_rounding_True_delay_1.0_2.0_seed_{}_spikes_inh-12503-0.dat'.format(spike_path, i),
                             skiprows=2,sep='\t')
    return exc, inh

In [6]:
spike_paths = ['C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_2\\brunel_rounding_1_2\\large_J',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_4\\brunel_rounding_1_2\\large_J',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_8\\brunel_rounding_1_2\\large_J',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_16\\brunel_rounding_1_2\\large_J',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_32\\brunel_rounding_1_2\\large_J',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_64\\brunel_rounding_1_2\\large_J',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_128\\brunel_rounding_1_2\\large_J',
               'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_256\\brunel_rounding_1_2\\large_J']

In [56]:
exc, inh = round_1_2(spike_paths[7])

In [57]:
cc_list_5 = get_all_ccs(exc,inh,bs=5)

In [58]:
cc_list_10 = get_all_ccs(exc,inh,bs=10)

In [59]:
cc_list_20 = get_all_ccs(exc,inh,bs=20)

In [60]:
cc_list_50 = get_all_ccs(exc,inh,bs=50)

In [61]:
cc_5 = pd.DataFrame(cc_list_5)
cc_10 = pd.DataFrame(cc_list_10)
cc_20 = pd.DataFrame(cc_list_20)
cc_50 = pd.DataFrame(cc_list_50)

In [62]:
cc_5.to_pickle('{}\cc_df_5.pkl'.format(spike_paths[7]))
cc_10.to_pickle('{}\cc_df_10.pkl'.format(spike_paths[7]))
cc_20.to_pickle('{}\cc_df_20.pkl'.format(spike_paths[7]))
cc_50.to_pickle('{}\cc_df_50.pkl'.format(spike_paths[7]))

And now for the qual delays. Must be done more by hans as delay values are different for each model resolution.

In [133]:
def equal():
    """
    Get spike data from brunel model with rounded spike times and delays drawn from discrete interval [1.0, 2.0].
    Send in any path we need.
    """
    spike_path = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_8\\brunel_continuous\\large_J'
    exc = {}
    for i in range(1,11):
        exc[i] = pd.read_csv(r'{}/brunel_continuous_delay_1.0_2.0_seed_{}_spikes_exc-12502-0.dat'.format(spike_path, i),
                             skiprows=2,sep='\t')

    inh = {}
    for i in range(1,11):
        inh[i] = pd.read_csv(r'{}/brunel_continuous__delay_1.0_2.0_seed_{}_spikes_inh-12503-0.dat'.format(spike_path, i),
                             skiprows=2,sep='\t')
    return exc, inh

In [134]:
exc, inh = equal()

In [135]:
cc_list_5 = get_all_ccs(exc,inh,bs=5)

In [136]:
cc_list_10 = get_all_ccs(exc,inh,bs=10)

In [137]:
cc_list_20 = get_all_ccs(exc,inh,bs=20)

In [138]:
cc_list_50 = get_all_ccs(exc,inh,bs=50)

In [139]:
cc_5 = pd.DataFrame(cc_list_5)
cc_10 = pd.DataFrame(cc_list_10)
cc_20 = pd.DataFrame(cc_list_20)
cc_50 = pd.DataFrame(cc_list_50)

In [140]:
p = 'C:\\Users\\Nida\\Documents\\NMBU\\master\\organized_spike_data\\resolution_1_8\\brunel_continuous\\large_J'
cc_5.to_pickle('{}\cc_df_5.pkl'.format(p))
cc_10.to_pickle('{}\cc_df_10.pkl'.format(p))
cc_20.to_pickle('{}\cc_df_20.pkl'.format(p))
cc_50.to_pickle('{}\cc_df_50.pkl'.format(p))

# Microcircuit
Get cc for all neurons microcircuit model. We will calculate it as the correlation between the firs tneuron in the group and all other neurons. 

In [14]:
def get_ccs(data,bs=5):
    spike_data = data.sort_values(by='time_ms')
    grouped = spike_data.groupby(spike_data['sender'])
    spike_trains = []
    for name, group in grouped:
        """
        The groups come in order, so that the first group is the first neuron in the population.
        """
        t = np.asarray(group['time_ms'])
        spiketrain = neo.core.SpikeTrain(t * pq.ms, t_start=0*pq.ms, t_stop=10000*pq.ms)
        spike_trains.append(spiketrain)
    """
    Now that we have all the spike trains, we can calculate the correlations between the first neuron in the population,
    and the rest.
    """
    correlations = []
    for i in range(len(spike_trains)):
        # could make sense to let range go from 1 and up, not include cc of first neruon with itself. But nice to have
        # same length for this statistic as the others. And all models will have this so okay.
        
        # calculate cc matrix for the pair of spiketrains
        cc_matrix = correlation_coefficient(BinnedSpikeTrain([spike_trains[i],spike_trains[0]], bin_size=bs*pq.ms))
        # retrieve element not on diagonal
        cc = cc_matrix[1,0]
        correlations.append(cc)
    return correlations

In [19]:
def create_cc_dfs(path):
    for p in path.iterdir():
        # resolution folder
        cc_list = list()
        for f in p.iterdir():
            if f.is_dir():
                data = pd.read_pickle(r'{}\spike_data.pkl'.format(f))
                ccs = get_ccs(data)
                cc_list.append(cvs)
        # iterated through all seed folders and calculated CV values.
        df = pd.DataFrame(cc_list) # save ccs to dataframe
        df.to_csv(r'{}\77175_cc5.csv'.format(p)) # save as csv file      

In [ ]:
path = Path('C:\\Users\\Nida\\Documents\\NMBU\\master\\microcircuit_spike_data\\droop')
create_cc_dfs(path)

path = Path('C:\\Users\\Nida\\Documents\\NMBU\\master\\microcircuit_spike_data\\equal')
create_cc_dfs(path)

path = Path('C:\\Users\\Nida\\Documents\\NMBU\\master\\microcircuit_spike_data\\continuous')
create_cc_dfs(path)